In [28]:
import os
import csv
import sys
import re

from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic
import numpy as np
from surprise.model_selection import train_test_split
import pprint

In [2]:
from surprise import SVD
from collections import defaultdict

In [3]:
import json
import pandas as pd
booksdf = pd.read_json("book-crossing_books.json", lines=True)
#df.head()
usersdf = pd.read_json("book-crossing_users.json", lines=True)
#df2.head()
explicitRatingsdf = pd.read_json("book-crossing_explicit_ratings.json", lines=True)
#df3.head()
implicitRatingsdf = pd.read_json("book-crossing_implicit_ratings.json", lines=True)
#df4.head()

In [4]:
explicitRatingsdf.head()

,book_id,implicit,rating,user_id
0,0155061224,NaN,5,276726
1,052165615X,NaN,3,276729
2,0521795028,NaN,6,276729
3,038550120X,NaN,7,276744
4,0060517794,NaN,9,276747


In [5]:
explicitRatingsdf.drop(['implicit'], axis=1)

,book_id,rating,user_id
0,0155061224,5,276726
1,052165615X,3,276729
2,0521795028,6,276729
3,038550120X,7,276744
4,0060517794,9,276747
5,0671537458,9,276747
6,0679776818,8,276747
7,0943066433,7,276747
8,1885408226,7,276747
9,0747558167,6,276748


In [6]:
explicitRatingsdf = explicitRatingsdf[['user_id', 'book_id', 'rating']]

In [7]:
explicitRatingsdf.head()
explicitRatingsdf.to_csv('file_name.csv', sep='\t', index=False)

In [8]:
explicitRatingsdf.head()

,user_id,book_id,rating
0,276726,0155061224,5
1,276729,052165615X,3
2,276729,0521795028,6
3,276744,038550120X,7
4,276747,0060517794,9


In [9]:
reader = Reader(line_format='user item rating', sep='\t', skip_lines=1)

In [10]:
from surprise import Dataset
data = Dataset.load_from_file('file_name.csv', reader)

In [11]:
data

In [12]:
# trainSet = data.build_full_trainset()
trainSet, testset = train_test_split(data, test_size=.1)
# sim_options = {'name': 'cosine',
#                'user_based': False
#                }

# model = KNNBasic(sim_options=sim_options)
model = SVD()
model.fit(trainSet)
#simsMatrix = model.compute_similarities()

In [13]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [16]:
predictions = model.test(testset)
top_n = get_top_n(predictions, n=10)

In [ ]:
testSubject = '85'
k = 10

testUserInnerID = trainSet.to_inner_uid(testSubject)
kNeighbors = heapq.nlargest(k, testUserRatings, key=lambda t: t[1])

candidates = defaultdict(float)
for book_id, rating in kNeighbors:
    similarityRow = simsMatrix[book_id]
    for book_id, score in enumerate(similarityRow):
        candidates[innerID] += score * (rating / 5.0)
        
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1
    
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break